In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from tqdm import tqdm

import os
import sys
import time
import math
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.init as init

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load DenseNet model
densenet_model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)
densenet_model = densenet_model.to(device)

Using cache found in /home/codespace/.cache/torch/hub/pytorch_vision_v0.10.0
/home/codespace/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/codespace/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
# preapre data

# PARAMS
batch_size = 256
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
target = 1 #automobile

#CIFAR-10 image tensor mean and std
NORM_MEAN = [0.4914, 0.4822, 0.4465]
NORM_STD = [0.2023, 0.1994, 0.2010]


# TODO: should we apply the below transformations?
transform_train = transforms.Compose([
    #transforms.RandomCrop(32, padding=4),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=NORM_MEAN,
                         std=NORM_STD)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=NORM_MEAN,
                         std=NORM_STD)
])

transform_image = transforms.ToPILImage()

print('==> Preparing data..')
trainset = torchvision.datasets.CIFAR10(root='../../00_data/', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='../../00_data/', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import time


def train(net, epochs, batch_size, lr, reg, log_every_n=50, trainloader=None, testloader=None):
    print('==> Preparing data..')

    best_acc = 0
    start_epoch = 0

    criterion = nn.CrossEntropyLoss()

    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=reg)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[int(epochs*0.5), int(epochs*0.75)], gamma=0.1)

    global_steps = 0
    start = time.time()

    for epoch in range(start_epoch, epochs):
        print('\nEpoch: %d' % epoch)
        net.train()
        train_loss = 0
        correct = 0
        total = 0

        for batch_idx, (inputs, targets) in enumerate(trainloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            global_steps += 1

            if global_steps % log_every_n == 0:
                end = time.time()
                num_examples_per_second = log_every_n * batch_size / (end - start)
                print("[Step=%d]\tLoss=%.4f\tacc=%.4f\t%.1f examples/second"
                      % (global_steps, train_loss / (batch_idx + 1), (correct / total), num_examples_per_second))
                start = time.time()

        scheduler.step()

        net.eval()
        test_loss = 0
        correct = 0
        total = 0

        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(testloader):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

        num_val_steps = len(testloader)
        val_acc = correct / total
        print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))

        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "pretrained_densenet_model.pt")

In [6]:
epochs = 20
batch_size = 256
learning_rate = 0.001
weight_decay = 0.0005

train(net=densenet_model, epochs=epochs, batch_size=batch_size, lr=learning_rate, reg=weight_decay,
      trainloader=trainloader, testloader=testloader)

==> Preparing data..

Epoch: 0
[Step=50]	Loss=4.1591	acc=0.2595	2771.6 examples/second
[Step=100]	Loss=2.7506	acc=0.4068	3207.6 examples/second
[Step=150]	Loss=2.1912	acc=0.4809	3394.1 examples/second
Test Loss=0.8868, Test acc=0.6949
Saving...

Epoch: 1
[Step=200]	Loss=0.8313	acc=0.7061	1695.0 examples/second
[Step=250]	Loss=0.7931	acc=0.7261	3199.4 examples/second
[Step=300]	Loss=0.7619	acc=0.7375	3193.0 examples/second
[Step=350]	Loss=0.7470	acc=0.7434	3161.9 examples/second
Test Loss=0.7088, Test acc=0.7572
Saving...

Epoch: 2
[Step=400]	Loss=0.5839	acc=0.8008	1728.1 examples/second
[Step=450]	Loss=0.5551	acc=0.8079	3182.0 examples/second
[Step=500]	Loss=0.5604	acc=0.8064	3185.2 examples/second
[Step=550]	Loss=0.5558	acc=0.8074	3194.7 examples/second
Test Loss=0.6504, Test acc=0.7791
Saving...

Epoch: 3
[Step=600]	Loss=0.4608	acc=0.8428	1751.6 examples/second
[Step=650]	Loss=0.4405	acc=0.8491	3208.5 examples/second
[Step=700]	Loss=0.4331	acc=0.8500	3211.6 examples/second
[Step=750]